Dieses Jupyter Notebook dient als "Data-Engineering-Teil" für das Capstone-Projekt.

Das Ziel ist die Erstellung eines , "real-world" Datensatzes über die Unternehmen im Nasdaq 100. Der Datensatz soll später als Wissensgrundlage (Knowledge Base) für ein (Retrieval-Augmented Generation RAG) - Chatbot dienen. 

Das Skript ist in zwei Hauptphasen unterteilt:

1. Zuerst wird die Liste der 100 Unternehmen (Ticker) von der englischen Wikipedia-Seite für den "NASDAQ-100" Index gescrapt. Hierfür verwende ich request, um einen HTTP 403-Fehler zu umgehen. (Natürlich hätte ich mir die Ticker, auch einfach so nehmen können, aber auf diese Weise kann man die CSV ganz einfach aktuell halten und muss nicht per Hand checken ob ein neues Unternehmen dazugekommen ist)

2. Anschließend möchte ich durch diese Ticker-Liste iterieren. Für jeden einzelnen Ticker wird die yfinance-Bibliothek verwendet, um detaillierte Unternehmensdaten von Yahoo Finance abzurufen. Der Fokus liegt dabei auf der Extraktion der textuellen Geschäftsbeschreibung (longBusinessSummary), da dieser Text die semantische Grundlage für das RAG-System bildet.
Natürlich möchte ich aber auch die Daten des Aktienkurses scrapen!

Das Endprodukt ist schließlich eine einzelne CSV-Datei (nasdaq_100_data.csv). Diese CSV wird dann durch vektorembedding zu einem für LLM´s lesbaren RAG inhalt. 

In [1]:
# Imports

import pandas as pd
import yfinance as yf # stellt die Verbindung zu Yahoo Finance her
import requests # Wikipedia blockiert einfache Anfragen. # requests' erlaubt, einen 'User-Agent' (eine Browser-Kennung) mitzusenden. Wikipedia denkt dann man sei ein normaler User.
import time # time.sleep(0.5) --> Wir haben ja gelernt, dass man beim Scraping nicht zu schnell hintereinander Anfragen schicken soll.
from transformers import pipeline  # Für Finbert! Die Pipeline Funktion übernimmt die folgenden Schritte für micht:  1: Das FinBERT-Modell  laden. 2: Den "Tokenizer" für FinBERT laden. 3: Den Nachrichtenteil in "Zahlen" umwandeln. 4: Die Zahlen durch das Modell laufen lassen. 5: Das Ergebnis in "Positiv", "Negativ" oder "Neutral" übersetzen.  

In [2]:
def get_nasdaq_100_tickers():
    """Holt die aktuelle Ticker-Liste des Nasdaq 100 von Wikipedia."""

    # Die URL der Wikipedia-Seite für den Nasdaq 100 
    url = "https://en.wikipedia.org/wiki/NASDAQ-100"
    
    # Header, um uns als normaler Browser zu identifizieren.
    # Ohne diesen Header blockiert Wikipedia die Anfrage mit Fehler 403. 
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:144.0) Gecko/20100101 Firefox/144.0" }  # diesen User-Agent bekommt man, wenn man einfach "what is my user agent" in Google eingibt!

    # HTML-Seite mit requests abrufen via headers
    response = requests.get(url, headers=headers)
    response.raise_for_status() # Zeigt einen Fehler, falls die Anfrage fehlschlägt (z.B. 404, 500)
        
    # Alle Tabellen auf der Seite mit pandas einlesen
    tables = pd.read_html(response.text) # pd.read_html extrahiert alle Tabellen aus dem HTML-Text. Pandas gibt dann eine Liste von DataFrames zurück.

    # Jetzt muss noch die richtige Tabelle gefunden werden. Das ist die Tabelle mit den Spalten 'Ticker' und 'Company'.
    nasdaq_df = None # None und nicht [] weil es ja ein DataFrame werden soll und keine Liste

    for t in tables:
        if "Ticker" in t.columns and "Company" in t.columns:  # Wenn die Tabelle t die Spalten 'Ticker' und 'Company' enthält, wird sie in nasdaq_df gespeichert.
            nasdaq_df = t
            break  # Schleife beenden
    return nasdaq_df # Muss natürlich noch etwas zurückgeben

# Funktion ausführen
nasdaq_tickers = get_nasdaq_100_tickers()

# Die ersten 10 Ticker zur Kontrolle ausgeben
nasdaq_tickers.head(10)


C:\Users\Dell\AppData\Local\Temp\ipykernel_6944\2535315252.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text) # pd.read_html extrahiert alle Tabellen aus dem HTML-Text. Pandas gibt dann eine Liste von DataFrames zurück.


,Ticker,Company,ICB Industry[14],ICB Subsector[14]
0,ADBE,Adobe Inc.,Technology,Computer Software
1,AMD,Advanced Micro Devices,Technology,Semiconductors
2,ABNB,Airbnb,Consumer Discretionary,Diversified Commercial Services
3,GOOGL,Alphabet Inc. (Class A),Technology,Computer Software
4,GOOG,Alphabet Inc. (Class C),Technology,Computer Software
5,AMZN,Amazon,Consumer Discretionary,Catalog/Specialty Distribution
6,AEP,American Electric Power,Utilities,Electric Utilities
7,AMGN,Amgen,Health Care,Biotechnology
8,ADI,Analog Devices,Technology,Semiconductors
9,AAPL,Apple Inc.,Technology,Computer Manufacturing


In [3]:
# Ich habe jetzt alle Ticker des Nasdaq 100. Nicht aber habe ich den Ticker zum Nasdaq selbst. Das ändere ich jetzt manuell. Ein bekannter Ticker für den nasdaq 100 ETF ist QQQ. 

# erstellt einen DataFrame für den Nasdaq ETF mit den gleichen Spalten wie nasdaq_tickers
nasdaq_etf_df = pd.DataFrame([{
    "Ticker": "QQQ",
    "Company": "Invesco QQQ Trust",  # Der offizielle Name für QQQ
    "ICB Subsector[14]": "ETF",
    "ICB Industry[14]": "ETF"
    }])

# jetzt einfach concatenieren
nasdaq_tickers = pd.concat([nasdaq_tickers, nasdaq_etf_df], ignore_index=True)
nasdaq_tickers.tail(5)  # Kontrolle, ob der ETF hinzugefügt wurde

,Ticker,Company,ICB Industry[14],ICB Subsector[14]
98,WBD,Warner Bros. Discovery,Consumer Discretionary,Miscellaneous Amusement & Recreation Services
99,WDAY,"Workday, Inc.",Technology,EDP Services
100,XEL,Xcel Energy,Utilities,Power Generation
101,ZS,Zscaler,Technology,EDP Services
102,QQQ,Invesco QQQ Trust,ETF,ETF


In [4]:
# Jetzt möchte ich diese Ticker nutzen um die Daten aus Yahoo Finance zu holen.

def get_yahoo_data(nasdaq_tickers):
    """
    Holt Unternehmensdaten und Kursinformationen etc.. für jeden Ticker aus Yahoo Finance.
    """
    data_list = [] # leere Liste, in die die Daten für jeden Ticker gespeichert werden

    for ticker in nasdaq_tickers["Ticker"]: # For schleife über alle Ticker im DataFrame nasdaq_tickers
        stock = yf.Ticker(ticker) # Erstellt ein yfinance.Ticker-Objekt für jedes Ticker-Symbol in der Schleife. Dadurch kann man Methoden wie .info oder .history() verwenden.

        # stock infos abrufen
        info = stock.info  # enthält viele Metadaten von Yahoo Finance z.B. auch current price
        
        # Stock Werte abrufen
        hist = stock.history(period="5y", interval="1mo")  # Holt historische Kursdaten der letzten 5Jahre (Open, High, Low, Close, Volume, ggf. Dividende/Splits)
        
        one_year_return = None  # initialisieren der Variablen als none --> werden noch berechnet
        volatility = None
        avg_monthly_return = None
        
        if not hist.empty: # Es werden nur Berechnungen gemacht, wenn tatsächlich Kursdaten vorhanden sind.
            # Rendite 1 Jahr
            if len(hist) >= 12: # Es müssen 12 Einträge für 12 Monate vorhanden sein!
                one_year_return = (hist["Close"].iloc[-1] - hist["Close"].iloc[-12]) / hist["Close"].iloc[-12]
                #  hist["Close"].iloc[-1] = aktueller Kurs. hist["Close"].iloc[-12] = Kurs von vor 12 Monaten  # Daraus wird die Rendite berechnet. 

            # Volatilität über 1 Jahr (Standardabweichung der monatlichen Renditen)
            # monatlichen Renditen
            monthly_returns = hist["Close"].pct_change().dropna() # hist["Close"] = Schlusskurse (monatlich, .pct_change() → prozentuale Veränderung zum Vormonat: (P_t - P_{t-1}) / P_{t-1}, .dropna() → entfernt den ersten Wert, der NaN ist, da es für den ersten Monat keine vorherige Kursänderung gibt
            # Volatilität der Aktie über das letzte Jahr
            volatility = monthly_returns.std() # Die Volatilität wird über die letzten 5 jahre berechnet
            # Durchschnittlicher monthly_return über 5 Jahre
            avg_monthly_return = monthly_returns.mean()


        time.sleep(0.2)  # 0.2 Sekunden warten, um die Server nicht zu überlasten
    
        # Relevante Felder aus info ziehen (mit get(), damit es keinen Fehler gibt, wenn ein Feld nicht existiert)
        data_list.append({  # Dictionary zur liste hinzufügen. --> Jedes Dictionary ist eine Zeile in der  Tabelle
            "Ticker": ticker,
            "Company": nasdaq_tickers.loc[nasdaq_tickers["Ticker"] == ticker, "Company"].values[0], # holt den Firmennamen aus dem DataFrame nasdaq_tickers, . values[0] um nur den String zu bekommen und nicht alles als Series
          # "ICB Sector": nasdaq_tickers.loc[nasdaq_tickers["Ticker"] == ticker, "ICB Sector"].values[0], # Das gleiche für ICB Sector ---> ich nehme lieber das von Yahoo
          # "ICB Industry": nasdaq_tickers.loc[nasdaq_tickers["Ticker"] == ticker, "ICB Industry"].values[0], # Das gleiche für ICB Industry --> ich nehme die Infos von Yahoo
            "Long Business Summary": info.get("longBusinessSummary"),
            "Sector (Yahoo)": info.get("sector"), # Das hier liefert die Gleiche Info  ICB Sector (Industry Classification Benchmark).Yahoo hat aber sein eigenes Klassifikationssystem...daher ist es etwas anders.
            "Industry (Yahoo)": info.get("industry"),  # Genauso. --> ich habe mich entschieden die Bezeichnungen von Yahoo zu nutzen.
            "Country": info.get("country"),# Land, in dem das Unternehmen seinen Sitz hat.
            "Market Cap": info.get("marketCap"), # Marktkapitalisierung, also wie viel das Unternehmen laut Börse insgesamt wert ist (Aktienkurs × Aktienanzahl)
            "Current Price": info.get("currentPrice"), # aktueller Aktienkurs (wie er auf Yahoo angezeigt wird)
            "52 Week High": info.get("fiftyTwoWeekHigh"), # Jahreshoch
            "Average Monthly Return": avg_monthly_return, # durchschnittliche monatliche rendite über 5 Jahre
            "Previous Close": info.get("previousClose"), # Schlusskurs vom vorherigen Handelstag.
            "Dividend Yield": info.get("dividendYield"),      # Dividendenrendite, zeigt, wie viel Dividende pro Jahr im Verhältnis zum Aktienkurs gezahlt wird
            "PE Ratio": info.get("trailingPE"),               # Kurs-Gewinn-Verhältnis der letzten 12 Monate (Price / Earnings), Bewertung der Aktie
            "Forward PE": info.get("forwardPE"),             # Erwartetes KGV für das nächste Jahr, basierend auf prognostiziertem Gewinn
            "PEG Ratio": info.get("pegRatio"),               # PE Ratio relativ zum Gewinnwachstum, zeigt Bewertung im Verhältnis zum Wachstum
            "Price to Book": info.get("priceToBook"),        # Kurs-Buchwert-Verhältnis, misst Marktwert im Verhältnis zum Buchwert des Unternehmens
            "Total Revenue": info.get("totalRevenue"),       # Gesamter Umsatz des Unternehmens (Revenue)
            "Debt to Equity": info.get("debtToEquity"),      # Verschuldungsgrad (Fremdkapital / Eigenkapital), zeigt finanzielle Stabilität
            "ROE": info.get("returnOnEquity"),               # Return on Equity, misst die Rentabilität des Eigenkapitals
            "1y Return": one_year_return,                    # Rendite über das letzte Jahr, berechnet aus historischen Kursen
            "Volatility": volatility,                         # Volatilität der Aktie über das letzte Jahr, gemessen als Standardabweichung der monatlichen Renditen
            "Website": info.get("website") # Unternehmenswebseite laut Yahoo Finance
        })

    # Alles in ein DataFrame packen
    df_yahoo = pd.DataFrame(data_list)
    return df_yahoo


# Funktion ausführen
nasdaq_yahoo_data = get_yahoo_data(nasdaq_tickers)

# Kontrolle
nasdaq_yahoo_data.tail()


,Ticker,Company,Long Business Summary,Sector (Yahoo),Industry (Yahoo),Country,Market Cap,Current Price,52 Week High,Average Monthly Return,...,PE Ratio,Forward PE,PEG Ratio,Price to Book,Total Revenue,Debt to Equity,ROE,1y Return,Volatility,Website
98,WBD,Warner Bros. Discovery,"Warner Bros. Discovery, Inc. operates as a med...",Communication Services,Entertainment,United States,57466707968,23.191,24.19,0.009436,...,122.057900,-178.392320,None,1.594870,3.786300e+10,89.857,0.00969,1.194040,0.178571,https://ir.wbd.com
99,WDAY,"Workday, Inc.","Workday, Inc. provides enterprise cloud applic...",Technology,Software - Application,United States,60112379904,225.140,294.00,0.002896,...,105.205605,27.125301,None,6.553912,8.959000e+09,41.169,0.06656,-0.127466,0.089770,https://www.workday.com
100,XEL,Xcel Energy,"Xcel Energy Inc., through its subsidiaries, en...",Utilities,Utilities - Regulated Electric,United States,47127973888,79.670,83.01,0.007367,...,24.289635,20.801567,None,2.224611,1.422800e+10,167.891,0.09449,0.219137,0.057947,https://mn.my.xcelenergy.com/s/
101,ZS,Zscaler,"Zscaler, Inc. operates as a cloud security com...",Technology,Software - Infrastructure,United States,43534307328,275.010,336.99,0.014132,...,NaN,77.250010,None,24.195848,2.673115e+09,99.850,-0.02699,0.524361,0.135175,https://www.zscaler.com
102,QQQ,Invesco QQQ Trust,To maintain the correspondence between the com...,None,None,None,231956512768,NaN,637.01,0.012904,...,33.271630,NaN,None,1.649282,NaN,NaN,NaN,0.160446,0.057133,None


In [5]:
# So jetzt möchte ich noch alles in Euro umrechennen. 

# Aktuellen Wechselkurs (USD -> EUR) holen
# Wichtig: "USDEUR=X" (wie viel EUR ist 1 USD wert, z.B. 0.93),
# NICHT "EURUSD=X" (wie viel USD ist 1 EUR wert, z.B. 1.08).
exchange_rate_usd_eur = yf.Ticker("USDEUR=X").info.get("regularMarketPrice")

# Liste der Spalten, die umgewandelt werden müssen. --> alle anderen Spalten sind Kennzahlen, Ratios & Prozentwerte, die Währungsunabhängig sind. Oder halt strings
cols_to_convert = ["Market Cap", "Current Price", "Previous Close", "Total Revenue"]

# Umrechnung durchführen
nasdaq_yahoo_data[cols_to_convert] = nasdaq_yahoo_data[cols_to_convert] * exchange_rate_usd_eur

# Kontrolle
nasdaq_yahoo_data.tail()

,Ticker,Company,Long Business Summary,Sector (Yahoo),Industry (Yahoo),Country,Market Cap,Current Price,52 Week High,Average Monthly Return,...,PE Ratio,Forward PE,PEG Ratio,Price to Book,Total Revenue,Debt to Equity,ROE,1y Return,Volatility,Website
98,WBD,Warner Bros. Discovery,"Warner Bros. Discovery, Inc. operates as a med...",Communication Services,Entertainment,United States,4.978916e+10,20.092682,24.19,0.009436,...,122.057900,-178.392320,None,1.594870,3.280450e+10,89.857,0.00969,1.194040,0.178571,https://ir.wbd.com
99,WDAY,"Workday, Inc.","Workday, Inc. provides enterprise cloud applic...",Technology,Software - Application,United States,5.208137e+10,195.061296,294.00,0.002896,...,105.205605,27.125301,None,6.553912,7.762077e+09,41.169,0.06656,-0.127466,0.089770,https://www.workday.com
100,XEL,Xcel Energy,"Xcel Energy Inc., through its subsidiaries, en...",Utilities,Utilities - Regulated Electric,United States,4.083168e+10,69.026088,83.01,0.007367,...,24.289635,20.801567,None,2.224611,1.232714e+10,167.891,0.09449,0.219137,0.057947,https://mn.my.xcelenergy.com/s/
101,ZS,Zscaler,"Zscaler, Inc. operates as a cloud security com...",Technology,Software - Infrastructure,United States,3.771812e+10,238.268664,336.99,0.014132,...,NaN,77.250010,None,24.195848,2.315987e+09,99.850,-0.02699,0.524361,0.135175,https://www.zscaler.com
102,QQQ,Invesco QQQ Trust,To maintain the correspondence between the com...,None,None,None,2.009671e+11,NaN,637.01,0.012904,...,33.271630,NaN,None,1.649282,NaN,NaN,NaN,0.160446,0.057133,None


In [ ]:
# Jetzt möchte ich noch aktuelle News als Spalte einbinden. Dafür möchte ich mir den aktuellsten Titel für jeden Ticker holen und den Inhalt der News mit FinBERT analysieren. FinBert ist ein vortrainiertes NLP-Modell, das speziell für Finanztexte entwickelt wurde und Sentiment-Analysen durchführen kann.
# Das Ergebnis wird dann eine Bewertung der News enthalten (positiv, negativ oder neutral). Damit hat der RAG Chatbot ein weiteres Element für seine Antworten.
# die Idee mit FinBERT habe ich aus einem Skript, was eigentlich für Open Webui gedacht ist. Damit hatte vorher schonmal rumprobiert:
# https://openwebui.com/t/johnthellm/yahoo_finance 

# Diese Funktion holt die aktuellste Nachricht für jeden Ticker und analysiert den Titel mit FinBERT und gibt einen DataFrame mit Titel, Link, Sentiment und Confidencezurück.

def get_latest_news_sentiment(nasdaq_tickers): # Funktion wird definiert und nimmt den DataFrame nasdaq_tickers_QQQ als Eingabe.
    # Pipeline für Sentiment-Analyse-->  Die Pipeline Funktion übernimmt die folgenden Schritte für micht:  1: Das FinBERT-Modell  laden. 2: Den "Tokenizer" für FinBERT laden. 3: Den Nachrichtenteil in "Zahlen" umwandeln. 4: Die Zahlen durch das Modell laufen lassen. 5: Das Ergebnis in "Positiv", "Negativ" oder "Neutral" übersetzen.  
    sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")  #  "sentiment-analysis" ist der Aufgabentyp, den die Pipeline ausführen soll--> Bewertung, ob ein Text positiv, negativ oder neutral ist. ProsusAI/finbert = das vortrainierte  Modell von Huggingface.
    print("Modell geladen.")

    results_list = [] # leere Liste, in die die Ergebnisse gespeichert werden.

    for ticker in nasdaq_tickers["Ticker"]:   # Schleife über jeden Ticker in der Spalte "Ticker" des DataFrames
        stock = yf.Ticker(ticker) #  # Yahoo Finance Objekt für den jeweiligen Ticker erstellen
        news = stock.news # # Abrufen der aktuellen Nachrichten zum Ticker
        content = news[0]["content"] # Den Inhalt der ersten Nachricht nehmen (neueste Nachricht)
        news_summary = content.get("summary")
        title = content["title"] # Titel der Nachricht 
        link = content["canonicalUrl"]["url"]# Link der Nachricht 

        result = sentiment_pipeline(news_summary)[0]  # Sentiment-Analyse auf news_summary ausführen

        results_list.append({  # Ergebnisse in einem Dictionary speichern und zur Liste hinzufügen
            "News Summary": news_summary,
            "Ticker": ticker,
            "Latest_News_Title": title,
            "Latest_News_Link": link,
            "Sentiment": result["label"],  # Sentiment-Label (Positiv, Negativ, Neutral)
            "Confidence": result["score"]  # Wahrscheinlichkeitswert des Sentiments der Analyse
        })

        time.sleep(0.2) # Kleine Pause zwischen den Anfragen

    return pd.DataFrame(results_list)  # Am Ende alle Ergebnisse als pandas DataFrame zurückgeben


# Aufruf der Funktion mit dem DataFrame 
news_sentiment_data = get_latest_news_sentiment(nasdaq_tickers)  # Es kann sein, das ein Fehler passiert, weil es mal keine News gab für eine Aktie. Manchmal muss man diesen Block auch einfach nocmal ausführen. 

# Ergebnis anzeigen
news_sentiment_data.head(10) 

Device set to use cpu


Modell geladen.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Exception ignored in: <function tqdm.__del__ at 0x000001A024875BC0>
Traceback (most recent call last):
  File "c:\Users\Dell\anaconda3\envs\SMA\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\Dell\anaconda3\envs\SMA\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm' object has no attribute 'disp'


,News Summary,Ticker,Latest_News_Title,Latest_News_Link,Sentiment,Confidence
0,Adobe (ADBE) shares edged up in recent trading...,ADBE,Adobe (ADBE): Evaluating Value After Recent Sh...,https://finance.yahoo.com/news/adobe-adbe-eval...,positive,0.911973
1,Investors will be tracking retail earnings and...,AMD,"Economic data returns, retail earnings feature...",https://finance.yahoo.com/news/economic-data-r...,negative,0.648860
2,Large-cap stocks have the power to shape entir...,ABNB,3 Large-Cap Stocks Worth Investigating,https://finance.yahoo.com/news/3-large-cap-sto...,neutral,0.906516
3,Futures rose Monday. Google popped amid Salesf...,GOOGL,Stock Market Today: Dow Jones Rises; Google Po...,https://www.investors.com/market-trend/stock-m...,negative,0.925637
4,Futures rose Monday. Google popped amid Salesf...,GOOG,Stock Market Today: Dow Jones Rises; Google Po...,https://www.investors.com/market-trend/stock-m...,negative,0.925637
5,"​Amazon.com, Inc. (NASDAQ:AMZN) is one of the ...",AMZN,Bank of America Securities Reiterates a Buy on...,https://finance.yahoo.com/news/bank-america-se...,neutral,0.768268
6,"Utility companies like NRG Energy, Constellati...",AEP,5 Utility Stocks Outperforming The Market,https://finance.yahoo.com/news/5-utility-stock...,positive,0.786558
7,Novo Nordisk reported Monday that a Ozempic pi...,AMGN,Novo Nordisk Dives As Ozempic Fails To Slow Al...,https://www.investors.com/news/novo-nordisk-di...,negative,0.934518
8,Investors will be tracking retail earnings and...,ADI,"Economic data returns, retail earnings feature...",https://finance.yahoo.com/news/economic-data-r...,negative,0.648860
9,"Replacing Tim Cook, who has been at Apple sinc...",AAPL,"Apple’s presumptive future CEO, John Ternus, h...",https://finance.yahoo.com/news/apples-presumpt...,negative,0.499873


In [8]:
# Jetzt muss ich nur noch die beiden DataFrames mergen. Damit ich eine Finale CSV erstellen kann.
# Merge der beiden DataFrames über die Spalte "Ticker"
final_df = pd.merge(
    nasdaq_yahoo_data, 
    news_sentiment_data, 
    on="Ticker", 
    how="outer"
)

# Optional: Spalten anzeigen
final_df.head(15)

,Ticker,Company,Long Business Summary,Sector (Yahoo),Industry (Yahoo),Country,Market Cap,Current Price,52 Week High,Average Monthly Return,...,Debt to Equity,ROE,1y Return,Volatility,Website,News Summary,Latest_News_Title,Latest_News_Link,Sentiment,Confidence
0,AAPL,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",Technology,Consumer Electronics,United States,3.490741e+12,235.218936,277.32,0.015119,...,152.411,1.71422,0.089042,0.071134,https://www.apple.com,"Replacing Tim Cook, who has been at Apple sinc...","Apple’s presumptive future CEO, John Ternus, h...",https://finance.yahoo.com/news/apples-presumpt...,negative,0.499873
1,ABNB,Airbnb,"Airbnb, Inc., together with its subsidiaries, ...",Consumer Cyclical,Travel Services,United States,6.060818e+10,98.994864,163.93,-0.002326,...,26.458,0.30776,-0.130508,0.108444,https://www.airbnb.com,Large-cap stocks have the power to shape entir...,3 Large-Cap Stocks Worth Investigating,https://finance.yahoo.com/news/3-large-cap-sto...,neutral,0.906516
2,ADBE,Adobe Inc.,Adobe Inc. operates as a technology company wo...,Technology,Software - Application,United States,1.191485e+11,280.878216,557.90,-0.002160,...,56.542,0.52875,-0.270959,0.102336,https://www.adobe.com,Adobe (ADBE) shares edged up in recent trading...,Adobe (ADBE): Evaluating Value After Recent Sh...,https://finance.yahoo.com/news/adobe-adbe-eval...,positive,0.911973
3,ADI,Analog Devices,"Analog Devices, Inc. engages in the design, ma...",Technology,Semiconductors,United States,9.902180e+10,201.282048,258.13,0.011878,...,25.541,0.05652,0.112287,0.074724,https://www.analog.com,Investors will be tracking retail earnings and...,"Economic data returns, retail earnings feature...",https://finance.yahoo.com/news/economic-data-r...,negative,0.648860
4,ADP,Automatic Data Processing,"Automatic Data Processing, Inc. provides cloud...",Technology,Software - Application,United States,8.883741e+10,219.303168,329.93,0.009856,...,154.917,0.70573,-0.117273,0.063002,https://www.adp.com,Automatic Data Processing (ADP) shares have sh...,Assessing ADP (ADP) Valuation After Recent Sha...,https://finance.yahoo.com/news/assessing-adp-a...,negative,0.903957
5,ADSK,Autodesk,"Autodesk, Inc. provides 3D design, engineering...",Technology,Software - Application,United States,5.390314e+10,251.949120,329.09,0.003260,...,100.700,0.40200,-0.016138,0.091454,https://www.autodesk.com,Investors will be tracking retail earnings and...,"Economic data returns, retail earnings feature...",https://finance.yahoo.com/news/economic-data-r...,negative,0.648860
6,AEP,American Electric Power,"American Electric Power Company, Inc., an elec...",Utilities,Utilities - Regulated Electric,United States,5.604005e+10,104.695776,124.80,0.010995,...,152.132,0.12923,0.355863,0.056071,https://www.aep.com,"Utility companies like NRG Energy, Constellati...",5 Utility Stocks Outperforming The Market,https://finance.yahoo.com/news/5-utility-stock...,positive,0.786558
7,AMAT,Applied Materials,"Applied Materials, Inc. engages in the provisi...",Technology,Semiconductor Equipment & Materials,United States,1.546107e+11,194.077932,242.50,0.022726,...,32.109,0.35508,0.391178,0.108862,https://www.appliedmaterials.com,Wondering if Applied Materials is a smart buy ...,Assessing Applied Materials After Recent Partn...,https://finance.yahoo.com/news/assessing-appli...,negative,0.596433
8,AMD,Advanced Micro Devices,"Advanced Micro Devices, Inc. operates as a sem...",Technology,Semiconductors,United States,2.874389e+11,176.554992,267.08,0.026300,...,6.366,0.05317,0.687060,0.166779,https://www.amd.com,Investors will be tracking retail earnings and...,"Economic data returns, retail earnings feature...",https://finance.yahoo.com/news/economic-data-r...,negative,0.648860
9,AMGN,Amgen,"Amgen Inc. discovers, develops, manufactures, ...",Healthcare,Drug Manufacturers - General,United States,1.574758e+11,292.444656,345.84,0.011494,...,567.491,0.81710,0.337197,0.068791,https://www.amgen.com,Novo 

In [9]:
# csv speichern
final_df.to_csv("nasdaq_100_final_for_RAG.csv", index=False)

In [10]:
final_df.columns


Index(['Ticker', 'Company', 'Long Business Summary', 'Sector (Yahoo)',
       'Industry (Yahoo)', 'Country', 'Market Cap', 'Current Price',
       '52 Week High', 'Average Monthly Return', 'Previous Close',
       'Dividend Yield', 'PE Ratio', 'Forward PE', 'PEG Ratio',
       'Price to Book', 'Total Revenue', 'Debt to Equity', 'ROE', '1y Return',
       'Volatility', 'Website', 'News Summary', 'Latest_News_Title',
       'Latest_News_Link', 'Sentiment', 'Confidence'],
      dtype='object')